## Intro

The purpose of today's class is to explore data using **interactive visualizations**. Interactivity is a key part of modern dataviz. It's a way to allow users of your visualizations get their own feel for the data ... to create richer visualization, where people who use your work can expose more of the data by exploring.


## Part 1: Video Lectures and Reading

Starting this week, we'll be playing around with *explanatory data visualization*. Roughly speaking this means using data visualization to communicate your results to others. Thus, there are new things to think about. 

Until today we have worked with static data visualization. However, exploratory data analysis means to be able to explore the multi-faceted nature of data and *interactive dataviz* is a handy tool to do it! It allows to play with the data: Toggle the view. Zoom. Drag. Show more details. All those things. Those are a key part of modern data visualization. 

The video below provides context about these points.

We start with the video and then read a bit from a scientific article about types of explanatory dataviz. (*The video is from an old version of the class that used D3, so just ignore those parts.*).

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/yHKYMGwefso/0.jpg)](https://www.youtube.com/watch?v=yHKYMGwefso)

> *Exercises*: Explanatory data visualization
> * What are the three key elements to keep in mind when you design an explanatory visualization?
> * In the video I talk about (1) *overview first*,  (2) *zoom and filter*,  (3) *details on demand*. 
>   - Go online and find a visualization that follows these principles (don't use one from the video). 
>   - Explain how your video achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.
> * Explain in your own words: How is explanatory data analysis different from exploratory data analysis?

## Part 2: Interactive visualizations with Bokeh



To really master interactive visualizations, you will need to work with JavaScript, especially [D3](https://d3js.org). Given the limited time available for this class, we can't squeeze that in. But luckily Python has some pretty good options for interactive visualizations. You can find a range of different options [here](https://mode.com/blog/python-interactive-plot-libraries/).

Today, we'll explore [`Bokeh`](https://docs.bokeh.org/en/latest/), which provides lots of nice interactive funtionalities to Python. To work with Bokeh, we first set up our system:

1. If you haven't installed it yet please do so. You can simply follow [these steps](https://docs.bokeh.org/en/latest/docs/first_steps/installation.html)

2. To include Bokeh in your notebooks you can follow the [Bokeh: Using with Jupyter](https://docs.bokeh.org/en/latest/docs/user_guide/output/jupyter.html#jupyter) guide. Come back to this one when you need it

3. We aim to give you a gentle start with Bokeh and I am going to include more example codes than usual in the follwing. However, to get a sense of how things work, I suggest you surf the web, find a Bokeh tutorial and scan through it. Also click around a bit in [the official docs](https://docs.bokeh.org/en/latest/docs/user_guide.html#userguide).

In [76]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, gridplot, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Legend
from bokeh.plotting import figure, show, output_file
from bokeh.models import FactorRange, LegendItem
output_notebook()

Loading BokehJS ...

Ok. Let's get started. First a general announcement on the data.

> **Announcement**
> * During this entire lecture, as always, we are going to work with the SF Crime Data. 
> * We will use data for the **period 2010-2017***.


Now, to get you in the mood here's a little gif to illustrate what the goal of this exercise is:

![Movie](https://github.com/suneman/socialdata2021/blob/main/files/week8_1.gif?raw=true)

In [117]:
# Henter datasæt
data = pd.read_csv("../1_opstart/Data/data_2003_to_May_2018.csv")
data["Dato"] = pd.to_datetime(data['Date'], format="%m/%d/%Y") # Make the data datetime
data['year']= data['Dato'].dt.year # Make a variable for year
data["week"]= data["Dato"].dt.day_name()
data["month"]= data["Dato"].dt.month_name()
data["Hour"] = pd.to_datetime(data["Time"], format="%H:%M").dt.hour
data["12_hours"] = [datetime.strftime(datetime.strptime(str(t), "%H"), "%I %p") for t in data["Hour"] ]
data["Weekday_Hour"] = data["week"] + "-" + data["12_hours"] 

# From 2010 to 2017
data = data[(data['Dato'].dt.year >= 2010) & (data['Dato'].dt.year <= 2017)]


weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
hour_order = [datetime.strftime(datetime.strptime(str(t), "%H"), "%I %p") for t in range(24)]
weekday_hour_order = [weekday + "-" + hour for weekday in weekday_order for hour in hour_order]
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 
                   'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT',
                    'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

data = data[data['Category'].isin(list(focuscrimes))]

In [118]:
columns = list(focuscrimes)
columns.insert(0,"Hour")
# create empty DataFrame with columns
data_prep_df = pd.DataFrame(columns=columns)

# Grouping the data
amount_hour = data.groupby(["Hour", "Category"]).size()


total_amount = data.value_counts("Category")

for i in range(24):
    crime_list = []
    crime_list.append(i)
    taller = 0
    for x in columns[1:]:
        try:
            normalised_value = amount_hour.loc[(i, x)]/total_amount.loc[(x)]
        except:
            normalised_value= 0
        crime_list.append(normalised_value)
        taller +=1
    print(taller)
    data_prep_df.loc[len(data_prep_df)] = crime_list


14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14



> ***Exercise***: Recreate the results from **Week 2** as an interactive visualisation (shown in the gif). To complete the exercise, follow the steps below to create your own version of the dataviz.

### Data prep

A key step is to set up the data right. So for this one, we'll be pretty strict about the steps. The workflow is

1. Take the data for the period of 2010-2017 and group it by hour-of-the-day.
2. We would like to be able to easily compare how the distribution of crimes differ from each other, not absolute numbers, so we will work on *normalized data*:
    * To normalise data for within a crime category you simply to devide the count for each hour by the total number of this crime type. (To give a concrete example in the `ASSAULT` category, take the number of assault-counts in 1st hour you should devide by the total number of assaults, then you devide number of assaults in 2nd hour by the total number of assaults and so on)
    *  Your life will be easiest if you organize your dataframe as shown in [this helpful screenshot](https://github.com/suneman/socialdataanalysis2020/blob/master/files/week8_1_data.PNG?raw=true).

If you've followed these steps, your data should be ready! Take a moment to celebrate. We now follow the [Bokeh guide for categorical data](https://docs.bokeh.org/en/latest/docs/user_guide/basic/bars.html):



1. First, let's convert our **Pandas Dataframe** to **Bokeh ColumnDataSource**: 
  > ```python
  > source = ColumnDataSource(your_processed_dataframe)
  > ## it is a standard way to convert your df to bokeh
  > ```
2. We also need to create an empty figure (we will add our stuff here later on). Mini sub-exercise: Find the a guide how to define a figure in Bokeh online. Here is a little help:
  > ```python
  > p = figure(...., x_range = FactorRange(factors=hours), ...) 
  > #p is a standard way to call figures in Bokeh
  > #make sure to add x_range. In my case hours is a list on the form ['1', '2', '3' ... , '24']
  > #read up on the FactorRange in the guide
  > #do not forget to add other attributes to the figure, e.g. title, axis names and so on
  > ```
3. Now we are going to add the bars. In order to do so, we will use **vbar** (see the guide for help):
  > ```python
  > bar ={} # to store vbars
  > ### here we will do a for loop:
  > for indx,i in enumerate(focuscrimes):
  >     bar[i] = p.vbar(x='name_of_the_column_that_contain_hours',  top=i, source= src, 
  >                     ### we will create a vbar for each focuscrime
  >                     legend_label=i,  muted_alpha=..., muted = ....) 
  > #i stands for a column that we use, top=y; we are specifying that our numbers comes from column i
  > #read up what legend_label, muted and muted_alpha do... you can add more attributes (you HAVE TO)
  > ```
4. The last thing to do is to make your legend interactive and display the figure:
  > ```python
  > p.legend.click_policy="mute" ### assigns the click policy (you can try to use ''hide'
  > show(p) #displays your plot
  > ```
5. You will notice that the legend appears in the middle of the figure (and it occludes some of the data). In order to fix this look into [this guide](https://stackoverflow.com/questions/26254619/position-of-the-legend-in-a-bokeh-plot) as a start. Below are some code snippets that you can use to deal with this problem (but read the guide first):
  > ```python
  > items = [] ### for the custom legend // you need to figure out where to add it
  > items.append((i, [bar[i]])) ### figure where to add it
  > legend = Legend(items=..., location=.....) ## figure where to add it
  > p.add_layout(...., ...) ## figure where to add it
  > ### if you read the guide, it will make sense
  > ```

Now you should be able to recreate this amazing visualisation.


**EXTRA feature**: If you're interested in detailed instructions for more Bokeh visualizations for your final project, you can find more inspiration **[here](https://github.com/suneman/socialdata2021/blob/main/lectures/Week8_extra_bokeh.ipynb)**.

In [134]:
from bokeh.layouts import column, gridplot
from bokeh.models import Legend, Div

source = ColumnDataSource(data_prep_df)

p = figure(x_axis_label="X-axis", 
           title="Crime plot", 
           y_axis_label="y_label",
           x_range=FactorRange(factors=[str(i) for i in range(0, 24)]),
           toolbar_location="above",
           width=800,
           height=600)

p.add_layout(legend, 'right')

items= []
bar = {}  # to store vbars
for indx, i in enumerate(columns[1:]):
    bar[i] = p.vbar(x='Hour', top=i, source=source,
                    legend_label=i, muted_alpha=0.5, muted=True)
    items.append((i, [bar[i]]))


legend = Legend(items=items, location=(10, 0))

p.legend.click_policy = "mute"

show(p)

## Part 3: Narrative Dataviz

Let's finish up with some reading

*Reading*: [Narrative Visualization: Telling Stories with Data](http://vis.stanford.edu/files/2010-Narrative-InfoVis.pdf) by Edward Segel and Jeffrey Heer. We'll read section 1-3 today. (And the rest a bit later).

When you get to section 3 it's fun to open up the examples mentioned by the authors in a browser and explore them as you read the text. 

> *Exercise*: Answer a couple of questions about the paper.
> 
> * What is the *Oxford English Dictionary's* defintion of a narrative?
> * What is your favorite visualization among the examples in section 3? Explain why in a few words.